In [34]:
import cx_Oracle
import os
import pandas as pd
#!pip insall cx_Oracle
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

ProgrammingError: Oracle Client library has already been initialized

In [35]:
connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

## new_data_input_to_table

1. 새로운 데이터 입력시 edit_mode에서 선택 가능
2. data_input_mode 를 만들어 해당 모드에서 진행한다.
    - edit_mode 에서만 선택가능, data_insert_mode 로 진행한다.
    - data_insert() 매서드로 db저장 및 xl_table 구현을 한다.
2. db상의 index 마지막 값을 찾아 그 이후 값부터 부여한다.
    - 사용자가 임의로 해당 셀에 값을 주었다고 하더라도 지우고 정해진 index값으로 넣는다.
3. data_input_mode에서는 table의 모든 값을 지운다.

### 과정
- data_insert_mode()
1. edit_mode 에서만 진행가능
    - 다른 모드에서는 edit_mode에서만 진행 가능함을 알림
2. table 내용 모두 삭제
3. status는 data_insert_mode로 변경


- data_input()
1. data 입력시작 
2. data 입력이 완료 되면, db에서 가장 높은 index값을 찾아 그값보다 높은 값을 순서대로 부여하여 db에 저장
3. db저장이 완료되면, 다시 data_insert_mode 종료 및 edit_mode 종료 
4. table에 다시 데이터를 불러들여온다.


In [50]:
sel_sht = wb_cy.selection.sheet
last_col_num =  sel_sht.range("XFD9").end('left').column

In [51]:
col_names = sel_sht.range((9,1),(9,last_col_num)).value
col_names

['SI_INDEX',
 'AWB_NO',
 'TRIP_NO',
 'SHIPMENT_NM',
 'NM_OF_PACKAGE',
 'INVOICE_DATE',
 'ORDER_NM',
 'ORDER_TOTAL',
 'UNIT_PRICE',
 'SHIP_TO',
 'ARRIVAL_DATE',
 'SHIP_DATE',
 'POD_DATE',
 'FOR_FREE',
 'REMARK',
 'PARCELS_NO',
 'COMMENT',
 'STATUS']

In [68]:
si_db = cursor.execute('select * from shipment_information')

In [69]:
df_si = pd.DataFrame(si_db.fetchall(),columns=col_names)
df_si

,SI_INDEX,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM,ORDER_TOTAL,UNIT_PRICE,SHIP_TO,ARRIVAL_DATE,SHIP_DATE,POD_DATE,FOR_FREE,REMARK,PARCELS_NO,COMMENT,STATUS
0,69,1041685812,None,None,1,None,None,None,None,삼성바이오로직스,2021-11-01,2021-11-02,SVC,None,None,None,삼성배송예정,None
1,70,3408190085,None,None,1,None,None,None,None,DHL특송,2021-11-02,2021-11-03,None,None,None,None,None,None
2,71,5379068192,None,None,1,None,None,None,None,DHL특송,2021-11-02,2021-11-03,None,None,None,None,None,None
3,72,1807999130,None,None,1,None,None,None,None,FTC,2021-11-09,2021-11-09,None,None,None,None,수리 후 FTC 택배발송,None
4,73,951869855693,106199286,123436073,1,None,2119970,None,None,서린바이오사이언스,2021-11-10,2021-11-10,2021-11-12,None,대리점,PYD002672682,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20482,19904,STO0165331,115379865,139820494,1,2022-10-11,2121926,313.3,None,동일과학,2022-10-19,2022-10-24,None,None,대리점,UPD3345577,None,None
20483,19905,STO0165331,115379865,139820494,None,2022-10-11,2123091,313.3,None,동일과학,2022-10-19,2022-10-24,None,None,대리점,UPD3345577,None,None
20484,19906,STO0165331,115379865,139820494,None,2022-10-11,2123205,313.3,None,동일과학,2022-10-19,2022-10-24,None,None,대리점,UPD3345577,None,None
20485,19907,STO0165331,115379865,139820494,1,2022-10-11,2123806,626.6,None,동일과학,2022-10-19,2022-10-24,None,None,대리점,UPD3345690,None,None


In [70]:
df_si = df_si.sort_values('SI_INDEX')
df_si = df_si.replace('None','')

In [71]:
df_si = df_si.reset_index(drop=True)
df_si= df_si.set_index("SI_INDEX")


In [72]:
sel_sht.range("A9").value = df_si